___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [1]:
import requests
link='https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange'
parametrs={'valcode': 'USD', 'date': '20211107', 'json': 'json'}

2. Отримати курс долара за останній рік.

In [2]:
import requests
from datetime import datetime, date, time, timedelta
import math

year_USD=[]
link='https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange'
parametrs={'valcode': 'USD', 'date': '20211107', 'json': 'json'}
date=datetime.now()
 
for i in range (0, 366):
    parametrs['date']=(date-timedelta(days=i)).strftime('%Y%m%d')
    response=requests.get(link, params=parametrs).json()
    year_USD.extend(response)

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [4]:
import requests
from datetime import datetime, date, time, timedelta
import math

year_USD=[]
link='https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange'
parametrs={'valcode': 'USD', 'date': '20211107', 'json': 'json'}
date=datetime.now()
 
for i in range (0, 366):
    parametrs['date']=(date-timedelta(days=i)).strftime('%Y%m%d')
    response=requests.get(link, params=parametrs).json()
    year_USD.extend(response)

month_avg=[]

for i in range(1, 12):
    avg=0
    sum_rate=0
    cnt=0
    otklon=0
    sum_r=0
    for j in year_USD:
        if int(datetime.strptime(j['exchangedate'], '%d.%m.%Y').month)==i:
            sum_rate+=int(j['rate'])
            cnt+=1
    avg=sum_rate/cnt
    for j in year_USD:
        if int(datetime.strptime(j['exchangedate'], '%d.%m.%Y').month)==i:
            sum_r+=(int(j['rate'])-avg)**2
            period=datetime.strptime(j['exchangedate'], '%d.%m.%Y').strftime('%Y-%m')
    otklon=math.sqrt(sum_r/cnt)
    month_avg.append({'період': period, 'середнє значення': avg,'відхилення': otklon})   

month_avg

[{'період': '2021-01',
  'середнє значення': 27.967741935483872,
  'відхилення': 0.17668469596940842},
 {'період': '2021-02',
  'середнє значення': 27.107142857142858,
  'відхилення': 0.30929478706587094},
 {'період': '2021-03',
  'середнє значення': 27.032258064516128,
  'відхилення': 0.17668469596940842},
 {'період': '2021-04',
  'середнє значення': 27.233333333333334,
  'відхилення': 0.42295258468165076},
 {'період': '2021-05', 'середнє значення': 27.0, 'відхилення': 0.0},
 {'період': '2021-06',
  'середнє значення': 26.933333333333334,
  'відхилення': 0.2494438257849295},
 {'період': '2021-07',
  'середнє значення': 26.838709677419356,
  'відхилення': 0.36779852422552833},
 {'період': '2021-08', 'середнє значення': 26.0, 'відхилення': 0.0},
 {'період': '2021-09',
  'середнє значення': 26.133333333333333,
  'відхилення': 0.339934634239519},
 {'період': '2021-10', 'середнє значення': 26.0, 'відхилення': 0.0},
 {'період': '2020-11',
  'середнє значення': 27.096774193548388,
  'відхиле

4. Дану інформацію записати у файл за допомогою pickle.

In [5]:
import pickle

with open('kurs.pickle', 'wb') as f:
    pickle.dump(month_avg, f)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [4]:
import requests
from bs4 import BeautifulSoup

import sqlite3

sql_text = """CREATE TABLE products_agrosem ( 
                                sku TEXT NOT NULL,
                                name TEXT NOT NULL,
                                price REAL  NULL, 
                                Weight INTEGER  NULL) """

conn = sqlite3.connect('products_agrosem.sqlite3')
curs = conn.cursor()
curs.execute(sql_text)
conn.close()

def insert_products (sku, name, price, Weight):
    sql_text = """INSERT INTO products_agrosem (sku, name, price, Weight) VALUES (?, ?, ?, ?)"""

    conn = sqlite3.connect('products_agrosem.sqlite3')
    curs = conn.cursor()
    curs.execute(sql_text, (sku, name, price, Weight))
    conn.commit()
    conn.close()
    

link = 'https://smallpacking.agrosem.ua/shop/'

#response = requests.get('https://smallpacking.agrosem.ua/shop/')

def get_soup(link):
    soup = None
    try:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'lxml')
    except Exception as e:
        print(e)   
    return soup

#print(get_soup(link))

def qty_of_pages(link):
    soup = get_soup(link)
    if soup != None:
        return soup.find('ul', class_='page-numbers').find_all('li')[-2].text
print(qty_of_pages(link))


def get_products (page):
    soup = get_soup(link+f'/page/{page}')
    if soup != None:
        for good in soup.find_all('div', class_="product-from-category-container"):
            href = good.a.get('href')
            soup_g = BeautifulSoup( requests.get(href).text, 'lxml')
            if soup_g != None:
                if soup_g.find('span', class_='sku').text==None:
                    sku='Null'
                else:
                    sku=soup_g.find('span', class_='sku').text
                if soup_g.find('div', class_='atributes-description').find('h2').text==None:
                    name='Null'
                else:
                    name=soup_g.find('div', class_='atributes-description').find('h2').text
                if soup_g.find('span', class_='regular-price').text==None:
                    price=0
                else:
                    if  soup_g.find('span', class_='regular-price').text.replace("грн", "").replace(" ", "")=='':
                        price=0
                    else:
                        try:
                            price=int(soup_g.find('span', class_='regular-price').text.replace("грн", "").replace(" ", "")) 
                        except Exception as e:
                            price=0
                if soup_g.find('tr', class_='woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_vaga')==None:
                    Weight=0
                else:
                    if  soup_g.find('tr', class_='woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_vaga').find('td', class_='woocommerce-product-attributes-item__value').text.replace("кг", "").replace(" ", "")=='':
                        Weight=0
                    else:
                        try:
                            Weight=int(soup_g.find('tr', class_='woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_vaga').find('td', class_='woocommerce-product-attributes-item__value').text.replace("кг", "").replace(" ", ""))
                        except Exception as e:
                            Weight=0
                insert_products (sku, name, price, Weight)


if __name__ == '__main__':
    qty  = int(qty_of_pages(link))
    for i in range(1, qty):
        get_products (i)                        
                            



20


### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [7]:
 import sqlite3

sql_text = """SELECT * FROM sqlite_master """

conn = sqlite3.connect('tysql.sqlite')
curs = conn.cursor()
info=curs.execute(sql_text)
print(info.fetchall())
conn.close()
   

[('table', 'Customers', 'Customers', 2, 'CREATE TABLE Customers\n(\n  cust_id      char(10)  NOT NULL ,\n  cust_name    char(50)  NOT NULL ,\n  cust_address char(50)  NULL ,\n  cust_city    char(50)  NULL ,\n  cust_state   char(5)   NULL ,\n  cust_zip     char(10)  NULL ,\n  cust_country char(50)  NULL ,\n  cust_contact char(50)  NULL ,\n  cust_email   char(255) NULL ,\n  PRIMARY KEY (cust_id)\n)'), ('index', 'sqlite_autoindex_Customers_1', 'Customers', 3, None), ('table', 'OrderItems', 'OrderItems', 4, 'CREATE TABLE OrderItems\n(\n  order_num  int          NOT NULL                      ,\n  order_item int          NOT NULL                      ,\n  prod_id    char(10)     NOT NULL                      ,\n  quantity   int          NOT NULL                      ,\n  item_price decimal(8,2) NOT NULL                      ,\n  PRIMARY KEY (order_num, order_item)                   ,\n  FOREIGN KEY (order_num) REFERENCES Orders (order_num) ,\n  FOREIGN KEY (prod_id) REFERENCES Products (prod

2. Виведіть перелік всіх таблиць.

In [16]:
import sqlite3

con = sqlite3.connect('tysql.sqlite')

def sql_fetch(con):
    cursorObj = con.cursor()
    cursorObj.execute('SELECT name from sqlite_master where type= "table"')
    print(cursorObj.fetchall())
    conn.close()
sql_fetch(con)

[('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]


3. Список всіх cust_id з таблиці Customers table.

In [17]:
import sqlite3

con = sqlite3.connect('tysql.sqlite')

cursorObj = con.cursor()
cursorObj.execute('SELECT cust_id from Customers')
print(cursorObj.fetchall())
con.close()

[('1000000001',), ('1000000002',), ('1000000003',), ('1000000004',), ('1000000005',)]


4. Всю таблицю Customers table

In [18]:
import sqlite3

con = sqlite3.connect('tysql.sqlite')

cursorObj = con.cursor()
cursorObj.execute('SELECT * from Customers')
print(cursorObj.fetchall())
con.close()

[('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com'), ('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None), ('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com'), ('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com'), ('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)]


5. Список клієнтів (cust_names) відсортованих від Z до A

In [21]:
import sqlite3
sql_text = """SELECT cust_name from Customers 
              order by cust_name desc """

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[('Village Toys',), ('The Toy Store',), ('Kids Place',), ('Fun4All',), ('Fun4All',)]


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [32]:
import sqlite3
sql_text = """SELECT a.cust_id, order_num from Customers a inner join Orders b on a.cust_id=b.cust_id
              order by a.cust_id, order_num """

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[('1000000001', 20005), ('1000000001', 20009), ('1000000003', 20006), ('1000000004', 20007), ('1000000005', 20008)]


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [33]:
import sqlite3
sql_text = """SELECT * from OrderItems
              order by quantity desc"""

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[(20009, 1, 'BNBG01', 250, 2.49), (20009, 2, 'BNBG02', 250, 2.49), (20009, 3, 'BNBG03', 250, 2.49), (20005, 1, 'BR01', 100, 5.49), (20005, 2, 'BR03', 100, 10.99), (20007, 2, 'BNBG01', 100, 2.99), (20007, 3, 'BNBG02', 100, 2.99), (20007, 4, 'BNBG03', 100, 2.99), (20007, 1, 'BR03', 50, 11.49), (20007, 5, 'RGAN01', 50, 4.49), (20006, 1, 'BR01', 20, 5.99), (20006, 2, 'BR02', 10, 8.99), (20006, 3, 'BR03', 10, 11.99), (20008, 3, 'BNBG01', 10, 3.49), (20008, 4, 'BNBG02', 10, 3.49), (20008, 5, 'BNBG03', 10, 3.49), (20008, 1, 'RGAN01', 5, 4.99), (20008, 2, 'BR03', 5, 11.99)]


8. Товар (з таблиці Products), ціна якого становить 9.49

In [35]:
import sqlite3
sql_text = """SELECT prod_id, prod_name from Products
              where prod_price=9.49"""

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[('RYL01', 'King doll'), ('RYL02', 'Queen doll')]


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [36]:
import sqlite3
sql_text = """SELECT prod_name, prod_price from Products
              where prod_price between 3 and 6
              order by prod_price"""

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[('Fish bean bag toy', 3.49), ('Bird bean bag toy', 3.49), ('Rabbit bean bag toy', 3.49), ('Raggedy Ann', 4.99), ('8 inch teddy bear', 5.99)]


10. Кількість товару, що було продано

In [37]:
import sqlite3
sql_text = """SELECT sum(quantity) as cnt from OrderItems"""

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[(1430,)]


11. Кількість найменувань товару, ціна якого більша за 4

In [39]:
import sqlite3
sql_text = """SELECT count(distinct prod_name) as cnt 
              from Products 
              where prod_price>4"""

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[(6,)]


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [45]:
import sqlite3
sql_text = """SELECT prod_name
              from Products 
              order by prod_price desc
              limit 3"""

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[('18 inch teddy bear',), ('King doll',), ('Queen doll',)]


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [48]:
import sqlite3
sql_text = """SELECT cust_id , (Select count(order_num) from Orders o where c.cust_id=o.cust_id) as cnt
              from Customers c
            """

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[('1000000001', 2), ('1000000002', 0), ('1000000003', 1), ('1000000004', 1), ('1000000005', 1)]


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [50]:
import sqlite3
sql_text = """SELECT c.cust_id ,  count(order_num) as cnt
              from Customers c left join Orders o on  c.cust_id=o.cust_id
              group by c.cust_id
            """

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[('1000000001', 2), ('1000000002', 0), ('1000000003', 1), ('1000000004', 1), ('1000000005', 1)]


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [51]:
import sqlite3
sql_text = """SELECT prod_name
              from Products 
              where prod_price<5
              union 
              SELECT prod_name
              from Products 
              where prod_price>=5
            """

con = sqlite3.connect('tysql.sqlite')
cursorObj = con.cursor()
cursorObj.execute(sql_text)
print(cursorObj.fetchall())

con.close()

[('12 inch teddy bear',), ('18 inch teddy bear',), ('8 inch teddy bear',), ('Bird bean bag toy',), ('Fish bean bag toy',), ('King doll',), ('Queen doll',), ('Rabbit bean bag toy',), ('Raggedy Ann',)]


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


In [52]:
1. 
Select model, speed, hd
FROM PC
WHERE price<500
----------------------------------------------
2.
Select DISTINCT maker
from Product
WHERE type='Printer'

----------------------------------------------
3.
Select model, ram,screen
from Laptop
Where price>1000

----------------------------------------------
4. 
Select *
from Printer
where color='y'
----------------------------------------------
5. 
Select model, speed, hd
From PC
WHERE CD in ('12x', '24x') AND price<600

----------------------------------------------
6. 
Select Distinct maker, speed
FROM Product Join Laptop ON Product.model=Laptop.model
Where hd>=10

----------------------------------------------
7. 
Select Product.model, PC.price

From Product JOIN PC ON Product.model=PC.model

WHERE maker='B'

UNION

Select Product.model, Laptop.price

From Product JOIN Laptop ON Product.model=Laptop.model

WHERE maker='B'

UNION

Select Product.model, Printer.price

From Product JOIN Printer ON Product.model=Printer.model

WHERE maker='B'

----------------------------------------------
8. 
Select DISTINCT maker
FROM Product
WHERE type='PC'

except
Select maker
From Product
Where type='Laptop'

----------------------------------------------
9. 
Select DISTINCT maker
FROM Product JOIN PC ON PC.model=Product.model
WHERE speed>=450

----------------------------------------------
10. 
Select model, price
From Printer
WHERE price=(Select Max (price) From Printer)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (Temp/ipykernel_1508/3468921937.py, line 2)